In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight



# Check if GPU is available
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# After mounting, verify access
!ls "/content/drive/My Drive/EEG_Project"

GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GAMEEMO  S01G1AllChannels.csv


In [17]:
!ls "/content/drive/MyDrive/EEG_Project"

GAMEEMO  S01G1AllChannels.csv


In [18]:
def create_windows(df, window_size=100, step_size=50):
    windows = []
    for i in range(0, len(df) - window_size + 1, step_size):
        window = df.iloc[i:i + window_size].values
        windows.append(window)
    return np.array(windows)

# Define emotions for each game
game_emotions = {
    1: 'calm',      # G1
    2: 'boring',    # G2
    3: 'funny',     # G3
    4: 'horror'     # G4
}

all_data = []
all_labels = []

# Load data
base_path = "/content/drive/MyDrive/EEG_Project"

for subject in range(1, 29):  # 28 subjects
    subject_str = f"{subject:02d}"
    for game in range(1, 5):  # 4 games
        path = f"/content/drive/My Drive/EEG_Project/GAMEEMO/(S{subject_str})/Preprocessed EEG Data/.csv format/S{subject_str}G{game}AllChannels.csv"
        try:
            game_data = pd.read_csv(path)
            if 'Unnamed: 14' in game_data.columns:
                game_data = game_data.drop(columns=['Unnamed: 14'])
            game_data = game_data.fillna(method='ffill')

            windows = create_windows(game_data)
            labels = [game_emotions[game]] * len(windows)

            all_data.append(windows)
            all_labels.extend(labels)
            print(f"Loaded game {game} for subject {subject_str}")
        except FileNotFoundError:
            print(f"Missing file for game {game}, subject {subject_str}")

# Prepare data
X = np.vstack(all_data)
y = np.array(all_labels)

# Convert labels to numeric
label_map = {'calm': 0, 'boring': 1, 'funny': 2, 'horror': 3}
y_numeric = np.array([label_map[label] for label in y])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_numeric, test_size=0.2, random_state=42)

<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')
<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')


Loaded game 1 for subject 01
Loaded game 2 for subject 01


<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')
<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')


Loaded game 3 for subject 01
Loaded game 4 for subject 01


<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')
<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')


...

<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')
<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')


Loaded game 1 for subject 28
Loaded game 2 for subject 28


<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')
<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')


Loaded game 3 for subject 28
Loaded game 4 for subject 28


<ipython-input-18-72b5c72dcafb>:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  game_data = game_data.fillna(method='ffill')


In [19]:


def create_hybrid_model(input_shape):
    model = Sequential([
        # CNN layers
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),

        Conv1D(filters=128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),

        # RNN layers
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(64),
        Dropout(0.2),

        Dense(32, activation='relu'),
        Dense(4, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:


print("\nTraining Hybrid Model...")
hybrid_model = create_hybrid_model((X_train.shape[1], X_train.shape[2]))
hybrid_history = hybrid_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)


Training Hybrid Model...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
 263/1651 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - accuracy: 0.3272 - loss: 1.3485